In [1]:
CUDA_VISIBLE_DEVICES=4

In [2]:
from datasets import load_dataset

ds = load_dataset("iamwarint/wongnai-restaurant-review")

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['review_body', 'stars'],
        num_rows: 19458
    })
    test: Dataset({
        features: ['review_body', 'stars'],
        num_rows: 500
    })
})

In [4]:
from datasets import concatenate_datasets

data = concatenate_datasets([ds['train'],ds['test']])

In [5]:
data

Dataset({
    features: ['review_body', 'stars'],
    num_rows: 19958
})

In [6]:
df = data.to_pandas()

In [7]:
df.sample(n=10, random_state=42)


,review_body,stars
971,ไก่ทอดเจ้าดัง เลยลองสั่งมาทาน โดยใช้ GRAP ส่งฟ...,4
606,หนึ่งในร้านดังสุดของเมืองพัทยา​ ร้านใหญ่มากๆ​...,3
7197,ไปบางปู ไปกินอะไรดี คำถามยอดฮิตพอค้นข้อมูลไปข้...,5
695,มารอบนี้เป็นครั้งที่สอง เพราะอยากทานปลาเก๋าที่...,4
18346,หลังจากที่ไม่ได้ทานข้าวมันไก่เป็นเวลานานมาก วั...,3
6935,ขอฝากเพจ world food critic ของผมด้วยนะครับ \nฝ...,3
6291,เป็นบรรยากาศเหมือนนั่งร้านข้างทางแต่ไฮโซ 555 \...,4
17526,มากินบุฟเฟต์ราคา 700++ กินได้ทั้ง ซาชิมิ ซูชิ ...,5
7479,ร้าน arno's butcher ตั้งอยู่บนย่านสาทร เป็นร้า...,3
11993,วันนี้ลองไปทานเรดซันคนเดียวโดยใช้คูปองไลน์ จาก...,2


In [8]:
df.to_csv("Raw_data.csv", index=False, encoding="utf-8-sig")
# ส่งออก data

In [ ]:
import pandas as 

df_binary = df.copy()
df_binary = df_binary[df_binary["stars"] != 3]  # ตัดรีวิว 3 ดาวออก


# map ดาวเป็น sentiment
df_binary["sentiment"] = df_binary["stars"].apply(lambda x: "negative" if x <= 2 else "positive")

# ตรวจสอบผลลัพธ์
df_binary.sample(n=10, random_state=42)


,review_body,stars,sentiment
99,อาหารทะเล ราคาไม่แพง บรรยากาศดีๆ \nไม่วุ่นวาย...,4,positive
3659,คือเราเคยกินสาขานาเกลือแล้ว1 ก็ยังไม่โดนใจเท่า...,5,positive
19249,-อาหาร(Food)⭐️⭐️⭐️⭐️⭐️\nจากเกียรติศัพท์เลื่องล...,5,positive
15562,ร้านอยู่ในซอย​ เหมือนเป็นบ้านแล้งนำมาเปิดเป็นร...,5,positive
7474,ร้านนี้ตามกระแสวงในมาจริงๆค่ะ ได้ยินว่าต้องจอง...,4,positive
15875,ได้ยินชื่อเสียงเค้กเปี่ยมสุขมาสักพักละ เพิ่งจะ...,4,positive
18435,ร้านเจ้าเก่าย่านดังสุดบางรักมิเชอลินไกด์ยกนิ้ว...,5,positive
17063,เราไปร้านนี้ตั้งแต่เริ่มเปิดใหม่ที่มีจำนวนโต๊ะ...,5,positive
17432,ตามชื่อร้านเลยคะ เบอร์เกอร์ เป็นตัวเด่นของทางร...,5,positive
8323,Chocolate Ville ร้านอาหารสไตล์ยุโรป มีการจัดตก...,5,positive


In [10]:
from sklearn.model_selection import train_test_split

# แบ่ง train (70%) และ test (30%)
train_df, temp_df = train_test_split(df_binary, test_size=0.3, random_state=42, shuffle=True)

# จาก test (30%) แบ่งครึ่งอีกทีเป็น val (10%) และ test (20%)
val_df, test_df = train_test_split(temp_df, test_size=2/3, random_state=42, shuffle=True)

print(f"Train: {len(train_df)}")
print(f"Val:   {len(val_df)}")
print(f"Test:  {len(test_df)}")


Train: 10890
Val:   1556
Test:  3112


In [11]:
train_path = "train_og_data.csv"
val_path   = "validate_data.csv"
test_path  = "test_data.csv"

train_df.to_csv(train_path, index=False, encoding="utf-8-sig")
val_df.to_csv(val_path, index=False, encoding="utf-8-sig")
test_df.to_csv(test_path, index=False, encoding="utf-8-sig")

In [12]:
train_df["sentiment"].value_counts()

sentiment
positive    10161
negative      729
Name: count, dtype: int64

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [ ]:
model_id = "scb10x/llama3.1-typhoon2-8b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda:4",
)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM

# ตรวจสอบจำนวน GPU ที่เห็น
print(f"จำนวน GPU ที่ระบบตรวจพบ: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")

# ตรวจสอบว่ามีโมเดลอยู่บน GPU ไหนบ้าง
devices = set(str(p.device) for p in model.parameters())
print(f"\nโมเดลถูกกระจายอยู่บน device: {devices}")

if hasattr(model, "hf_device_map"):
    print("\nDevice map ของโมเดล (ดูว่า layer ไหนอยู่ GPU ไหน):")
    for layer, dev in model.hf_device_map.items():
        print(f"   {layer:<40} → {dev}")
else:
    print("\nℹโมเดลไม่มี hf_device_map (อาจอยู่บน GPU เดียว)")

# ตรวจสอบการใช้หน่วยความจำปัจจุบันแต่ละ GPU
print("\nการใช้หน่วยความจำ (VRAM) ต่อ GPU:")
for i in range(torch.cuda.device_count()):
    allocated = torch.cuda.memory_allocated(i) / 1024**2
    reserved  = torch.cuda.memory_reserved(i) / 1024**2
    print(f"   GPU {i}: allocated={allocated:.1f} MB | reserved={reserved:.1f} MB")



🔹 จำนวน GPU ที่ระบบตรวจพบ: 5
   GPU 0: NVIDIA A100-SXM4-80GB
   GPU 1: NVIDIA A100-SXM4-80GB
   GPU 2: NVIDIA A100-SXM4-80GB
   GPU 3: NVIDIA A100-SXM4-80GB
   GPU 4: NVIDIA A100-SXM4-80GB

🔹 โมเดลถูกกระจายอยู่บน device: {'cuda:4'}

🔹 Device map ของโมเดล (ดูว่า layer ไหนอยู่ GPU ไหน):
                                            → cuda:4

🔹 การใช้หน่วยความจำ (VRAM) ต่อ GPU:
   GPU 0: allocated=0.0 MB | reserved=0.0 MB
   GPU 1: allocated=0.0 MB | reserved=0.0 MB
   GPU 2: allocated=0.0 MB | reserved=0.0 MB
   GPU 3: allocated=0.0 MB | reserved=0.0 MB
   GPU 4: allocated=15316.5 MB | reserved=15320.0 MB


In [16]:
def build_messages(sentiment: str, review: str):
    return [
        {
            "role": "system",
            "content": (
                "คุณคือนักเขียนรีวิวร้านอาหาร"
                "หน้าที่ของคุณคือการเขียนรีวิวใหม่ (rewrite) จากข้อความที่ให้มาให้มีลักษณะแตกต่างไปจากต้นฉบับ เช่น ปรับเปลี่ยนเนื้อหารีวิวหรือการเรียงลำดับเนื้อหา"
                "เขียนให้จบในย่อหน้าเดียวภายใน 70–110 คำ"
                "ถ้ามีอีโมจิให้ใช้ไม่เกิน 2 ตัว และต้องเหมาะกับอารมณ์รีวิว"
                "สะกดคำให้ถูกต้องตามหลักภาษาไทย"
                "**สำคัญ: ตอบเฉพาะข้อความรีวิว ห้ามมีคำว่า 'รีวิวใหม่:' หรือหัวข้ออื่นใดนำหน้า**"
            ),
        },
        {
            "role": "user",
            "content": (
                f"สร้างรีวิวใหม่โดยให้นายเขียนรีวิวขึ้นมาใหม่จากต้นฉบับ "
                f"(อารมณ์: {sentiment}) ซึ่งมีข้อความดังนี้:\n{review} "
                "ตอบเฉพาะข้อความรีวิวใหม่เท่านั้น"),
        },
    ]


In [ ]:
# Split train_df เป็น 5 ส่วน 
import numpy as np

num_shards = 5

# รีเซ็ต index ก่อน split
train_df_reset = train_df.reset_index(drop=True)

# แบ่งแบบขนาดใกล้เคียงกัน 5 ส่วน
shards = np.array_split(train_df_reset, num_shards)

# เซฟแต่ละ shard เป็นไฟล์ CSV สำหรับแต่ละ runpod
for i, shard_df in enumerate(shards, start=1):
    shard_path = f"train_shard_{i}.csv"
    shard_df.to_csv(shard_path, index=False, encoding="utf-8-sig")
    print(f"saved shard {i}: {len(shard_df)} rows -> {shard_path}")


saved shard 1: 2178 rows -> train_shard_1.csv
saved shard 2: 2178 rows -> train_shard_2.csv
saved shard 3: 2178 rows -> train_shard_3.csv
saved shard 4: 2178 rows -> train_shard_4.csv
saved shard 5: 2178 rows -> train_shard_5.csv


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
import pandas as pd
from tqdm import tqdm


# ตั้งค่าชิ้นงานของโน้ตบุ๊กนี้ 
SHARD_ID = 5  # โน้ตบุ๊กนี้รับผิดชอบ shard หมายเลขอะไร (1..5)

input_path  = f"train_shard_{SHARD_ID}.csv"
output_path = f"synthetic_shard_{SHARD_ID}.csv"

# โหลดข้อมูล shard นี้
chunk_df = pd.read_csv(input_path)

# สร้างข้อมูลสังเคราะห์ 
n_aug_per_original = 4
batch_size = 4

results = []

for _, row in tqdm(
    chunk_df.iterrows(),
    total=len(chunk_df),
    desc=f"Generating augmented reviews for shard {SHARD_ID}"
):
    review_text = row["review_body"]
    sent_label = row["sentiment"]

    remaining = n_aug_per_original

    while remaining > 0:
        k = min(batch_size, remaining)

        # เตรียม prompt k อัน (ให้โมเดลสุ่มเขียน k เวอร์ชัน)
        batch_messages = [build_messages(sent_label, review_text) for _ in range(k)]

        # สร้าง input_ids สำหรับโมเดล
        input_ids = tokenizer.apply_chat_template(
            batch_messages,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(model.device)

        # generate ทีเดียว ได้ k ชิ้น
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=220,
            do_sample=True,
            temperature=1.1,
            top_p=0.98,
        )

        # Decode
        decoded = tokenizer.batch_decode(
            outputs[:, input_ids.shape[-1]:],
            skip_special_tokens=True
        )

        # เก็บผลลัพธ์ลง results
        for text in decoded:
            results.append({
                "original_review": str(review_text).strip(),
                "review_body": str(text).strip(),
                "sentiment": str(sent_label).strip(),
                "is_synthesis": 1
            })

        remaining -= k

print(f"\n[Shard {SHARD_ID}] สร้างรีวิวสังเคราะห์ได้ {len(results)} แถว")

# ทำ DataFrame และเซฟเฉพาะสังเคราะห์ ─
synthetic_df = pd.DataFrame(results)

synthetic_df = synthetic_df[[
    "original_review",
    "review_body",
    "sentiment",
    "is_synthesis"
]]

synthetic_df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"[Shard {SHARD_ID}] saved synthetic only -> {output_path}")


Generating augmented reviews for shard 5:   0%|          | 0/2178 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Generating augmented reviews for shard 5:   0%|          | 1/2178 [00:04<2:48:16,  4.64s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Generating augmented reviews for shard 5:   0%|          | 2/2178 [00:09<3:02:52,  5.04s/it]T


[Shard 5] สร้างรีวิวสังเคราะห์ได้ 8712 แถว
[Shard 5] saved synthetic only -> synthetic_shard_5.csv


In [ ]:
import glob
synthetic_files = sorted(glob.glob("synthetic_shard_*.csv"))
print("พบไฟล์สังเคราะห์:", synthetic_files)

synthetic_all_list = []
for path in synthetic_files:
    df_tmp = pd.read_csv(path)
    synthetic_all_list.append(df_tmp)

synthetic_all_df = pd.concat(synthetic_all_list, ignore_index=True)

print("จำนวนรีวิวสังเคราะห์รวมทั้งหมด:", len(synthetic_all_df))
print(synthetic_all_df.head())

synthetic_all_df.to_csv("synthetic_all.csv", index=False, encoding="utf-8-sig")

พบไฟล์สังเคราะห์: ['synthetic_shard_1.csv', 'synthetic_shard_2.csv', 'synthetic_shard_3.csv', 'synthetic_shard_4.csv', 'synthetic_shard_5.csv']
จำนวนรีวิวสังเคราะห์รวมทั้งหมด: 43560
                                     original_review  \
0  ร้านญี่ปุ่นอร่อยขั้นเทพในซอยสุขุมวิท31 เน้นมาก...   
1  ร้านญี่ปุ่นอร่อยขั้นเทพในซอยสุขุมวิท31 เน้นมาก...   
2  ร้านญี่ปุ่นอร่อยขั้นเทพในซอยสุขุมวิท31 เน้นมาก...   
3  ร้านญี่ปุ่นอร่อยขั้นเทพในซอยสุขุมวิท31 เน้นมาก...   
4  สุกี้ช้างเผือก\n\nไม่ต้องพูดเยอะสำหรับร้านนี้ ...   

                                         review_body sentiment  is_synthesis  
0  ร้านอาหารญี่ปุ่นประยุกต์ในซอยสุขุมวิท 31 น่าสน...  positive             1  
1  ร้านอาหารญี่ปุ่นในซอยสุขุมวิท 31 นี่เป็นแหล่งร...  positive             1  
2  ประสบการณ์อาหารญี่ปุ่นที่น่าตื่นตาตื่นใจในร้าน...  positive             1  
3  ในซอยสุขุมวิท31 มีร้านญี่ปุ่นเล็กๆ ที่เสิร์ฟมา...  positive             1  
4  สุกี้ช้างเผือกคือร้านที่ต้องมาลองที่สุด โดยเฉพ...  positive             1  


In [ ]:
# Strain เดิม (รีวิวจริง)

original_df = train_df[["review_body", "sentiment"]].copy()

# เพิ่มคอลัมน์ให้ตรงกับสังเคราะห์
original_df["original_review"] = original_df["review_body"]
original_df["is_synthesis"] = 0

print("ขนาด train เดิม (original_df):", len(original_df))
print(original_df.head())



# STEP 3: รวมรีวิวจริง + รีวิวสังเคราะห์ เป็น final training set

augmented_full_df = pd.concat(
    [original_df, synthetic_all_df],
    ignore_index=True
)

print("ขนาดรวม (จริง + สังเคราะห์):", len(augmented_full_df))
print(augmented_full_df.head())



# บันทึกไฟล์ final ไว้สำหรับเอาไป fine-tune classifier

augmented_full_df.to_csv("train_augmented_full.csv", index=False, encoding="utf-8-sig")
print("บันทึกไฟล์ train_augmented_full.csv เรียบร้อย")

ขนาด train เดิม (original_df): 10890
                                             review_body sentiment  \
9408   ร้านญี่ปุ่นอร่อยขั้นเทพในซอยสุขุมวิท31 เน้นมาก...  positive   
15984  สุกี้ช้างเผือก\n\nไม่ต้องพูดเยอะสำหรับร้านนี้ ...  positive   
1852   ร้านขาหมูระดับตำนานของย่านสีลมนี้เปิดมาตั้งแต่...  positive   
17106  ใครๆ ก็บอกว่าต้องมาทาน เราก็ชอบผักเป็นทุนเดิม\...  positive   
6269   This place serves variety of Asian and Western...  positive   

                                         original_review  is_synthesis  
9408   ร้านญี่ปุ่นอร่อยขั้นเทพในซอยสุขุมวิท31 เน้นมาก...             0  
15984  สุกี้ช้างเผือก\n\nไม่ต้องพูดเยอะสำหรับร้านนี้ ...             0  
1852   ร้านขาหมูระดับตำนานของย่านสีลมนี้เปิดมาตั้งแต่...             0  
17106  ใครๆ ก็บอกว่าต้องมาทาน เราก็ชอบผักเป็นทุนเดิม\...             0  
6269   This place serves variety of Asian and Western...             0  
ขนาดรวม (จริง + สังเคราะห์): 54450
                                         review_body sentiment  \
0  